In [1]:
raw_data = """
13	North Tutupan	2024	2					316.9	1090.4			
14	South Tutupan	2024	2					56.2	685.0			
15	North Paringin	2024	2						210.9			
16	Wara I	2024	2					243.8	1057.0			
17	Wara II	2024	2						326.5			
18	Batubara Hamarung	2024	4					32.4	78.9			
19	Batubara Piyait	2024	5					20.5	57.2			
20	Batubara Tawahan	2024	6					65.7	203.6			
21	Batubara Benhes 3	2024	10						555.0			
22	Batubara Long Wehea 1	2024	11						1110.0			
23	Batubara Long Wehea 2	2024	12						23.0			
24	Batubara Long Wehea 4	2024	13						177.0			
25	Batubara Long Wehea 3	2024	14						949.0			
26	Batubara Marah Kenyah 2	2024	15						454.0			
27	Batubara Benhes 2	2024	16						4.3			
28	Batubara Gunung Kembang	2024	7					148.9	357.6			
29	Kestrel Mine	2024	17					165.5	416.3			
30	Batubara Melapeh Baru 1	2024	220					23.5	182.7			
31	Batubara Danum Paro	2024	219									
32	Bumbun Block	2024	22					55.5	174.5			
33	Juloi Northwest	2024	22						629.9			
34	Luon Block	2024	21					17.7	50.9			
35	Dahlia Arwana	2024	23					5.6	15.0			
36	Haju Block	2024	19					1.5	3.4			
37	Bara Block	2024	19					10.8	14.9			
38	Lumpunut Block	2024	20					86.1	94.2			
""".strip()

In [2]:
raw_data

'13\tNorth Tutupan\t2024\t2\t\t\t\t\t316.9\t1090.4\t\t\t\n14\tSouth Tutupan\t2024\t2\t\t\t\t\t56.2\t685.0\t\t\t\n15\tNorth Paringin\t2024\t2\t\t\t\t\t\t210.9\t\t\t\n16\tWara I\t2024\t2\t\t\t\t\t243.8\t1057.0\t\t\t\n17\tWara II\t2024\t2\t\t\t\t\t\t326.5\t\t\t\n18\tBatubara Hamarung\t2024\t4\t\t\t\t\t32.4\t78.9\t\t\t\n19\tBatubara Piyait\t2024\t5\t\t\t\t\t20.5\t57.2\t\t\t\n20\tBatubara Tawahan\t2024\t6\t\t\t\t\t65.7\t203.6\t\t\t\n21\tBatubara Benhes 3\t2024\t10\t\t\t\t\t\t555.0\t\t\t\n22\tBatubara Long Wehea 1\t2024\t11\t\t\t\t\t\t1110.0\t\t\t\n23\tBatubara Long Wehea 2\t2024\t12\t\t\t\t\t\t23.0\t\t\t\n24\tBatubara Long Wehea 4\t2024\t13\t\t\t\t\t\t177.0\t\t\t\n25\tBatubara Long Wehea 3\t2024\t14\t\t\t\t\t\t949.0\t\t\t\n26\tBatubara Marah Kenyah 2\t2024\t15\t\t\t\t\t\t454.0\t\t\t\n27\tBatubara Benhes 2\t2024\t16\t\t\t\t\t\t4.3\t\t\t\n28\tBatubara Gunung Kembang\t2024\t7\t\t\t\t\t148.9\t357.6\t\t\t\n29\tKestrel Mine\t2024\t17\t\t\t\t\t165.5\t416.3\t\t\t\n30\tBatubara Melapeh Baru 1\t2024\

In [ ]:
rows = []
for line in raw_data.splitlines():
    fields = line.split("\t")
    
    # Safely parse and convert fields
    def parse_float(val): return float(val) if val.strip() else None
    def parse_int(val): return int(val) if val.strip() else None
    def parse_str(val): return val.strip() if val.strip() else None

    row = {
        "id": parse_int(fields[0]),
        "name": parse_str(fields[1]),
        "year": parse_int(fields[2]),
        "company_id": parse_int(fields[3]),
        "calorific_value": parse_str(fields[4]),
        "production_volume": parse_float(fields[5]),
        "overburden_removal_volume": parse_float(fields[6]),
        "strip_ratio": parse_float(fields[7]),
        "reserve": parse_float(fields[8]),
        "resource": parse_float(fields[9]),
        "province": parse_str(fields[10]) if len(fields) > 10 else None,
        "city": parse_str(fields[11]) if len(fields) > 11 else None,
        "mineral_type": parse_str(fields[12]) if len(fields) > 12 else None,
    }
    rows.append(row)

In [6]:
from db import Company, MiningSite
from peewee import (
    Model,
    SqliteDatabase)

db = SqliteDatabase('coal-db.sqlite')

In [7]:
with db.atomic():
    for batch in [rows[i:i+50] for i in range(0, len(rows), 50)]:
        MiningSite.insert_many(batch).execute()

In [ ]:
import pandas as pd
from db import Company, MiningSite
from peewee import fn
import re
from tabulate import tabulate

In [2]:
companies = Company.select()
for company in companies:
    pass

ms = MiningSite.select()
for m in ms:
    pass

In [3]:
existing_companies = [re.sub(r'\b(PT|Tbk)\b', '', company.name).lower().strip() for company in companies]
# existing_companies = [company.name.lower().strip() for company in companies]

In [4]:
nameyearcompany_query = (
    MiningSite
    .select(
        Company.name,
        fn.CONCAT(Company.name, MiningSite.name, MiningSite.year).alias('nameyearcompany'),
        MiningSite  # selects all columns from MiningSite
    )
    .join(Company, on=(MiningSite.company == Company.id))
)
existing_nameyearcompany = [row.nameyearcompany for row in nameyearcompany_query]

In [5]:
c_df = pd.read_csv("Sample db - company_directory.csv")
ccp_df = pd.read_csv("Sample db - company_performance.csv")
ms_df = pd.read_csv("Sample db - mining_site.csv")

In [6]:
c_df.loc[c_df['company_type'] == "Owner", 'company_type'] = "Mine Owner"
excluded_companies = ("PT BUMA Internasional Grup Tbk", "PT Murawai Coal")
c_df = c_df[~c_df["name"].isin(excluded_companies)]
c_df['name_cleaned'] = c_df['name'].str.replace(r'\b(PT|Tbk)\b', '', regex=True).str.lower().str.strip()

In [7]:
c_df[~c_df['name_cleaned'].isin(existing_companies)]

,id,name,idx_ticker,parent company id,operation_province,operation_kabkot,representative_address,company_type,key_operation,activities,...,phone_number,email,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,name_cleaned
91,92,PT Adaro Indonesia,NaN,25.0,Kalimantan Selatan,"Tabalong, Balangan",NaN,Mine Owner,Mining Services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adaro indonesia
111,112,PT Anugerah Energi,NaN,50.0,Sumatera Selatan,NaN,NaN,Mine Owner,Mining,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anugerah energi
128,129,PT Rimba Perkasa Utama,NaN,NaN,NaN,NaN,NaN,Contractor,Mining Services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rimba perkasa utama


In [8]:
c_df = c_df[c_df['name_cleaned'].isin(existing_companies)]

In [9]:
cc_ccp_df = pd.merge(c_df[['id', 'name']], ccp_df[['company_id', 'mine_id']], left_on='id', right_on='company_id', how='inner')

In [ ]:
cc_ccp_ms_df = pd.merge(cc_ccp_df, ms_df, left_on='mine_id', right_on='mine_id', how='inner')
cc_ccp_ms_df = cc_ccp_ms_df[cc_ccp_ms_df['year_measured'].notna()]
cc_ccp_ms_df['year_measured'] = cc_ccp_ms_df['year_measured'].astype(int)
cc_ccp_ms_df['nameyearcompany'] = (
    cc_ccp_ms_df['name_x'].astype(str) +
    cc_ccp_ms_df['name_y'].astype(str) +
    cc_ccp_ms_df['year_measured'].astype(str)
)

In [11]:
new_cc_ccp_ms_df = cc_ccp_ms_df[~cc_ccp_ms_df['nameyearcompany'].isin(existing_nameyearcompany)]
new_cc_ccp_ms_df

,id_x,name_x,company_id,mine_id,id_y,name_y,year_measured,calorific_value,production_volume,overburden_removal_volume,strip_ratio,resources_inferred,resources_indicated,resources_measured,reserves_proved,reserves_probable,province,city,nameyearcompany
0,2,PT Fajar Sakti Prima,2,1.0,1,Tabang,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Kartanegara,PT Fajar Sakti PrimaTabang2022
1,3,PT Bara Tabang,3,1.0,1,Tabang,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Kartanegara,PT Bara TabangTabang2022
5,7,PT Brian Anjat Sentosa,7,7.0,7,BAS,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Kartanegara,PT Brian Anjat SentosaBAS2022
10,12,PT Perkasa Inakakerta,12,5.0,5,Perkasa Inakakerta,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Timur,PT Perkasa InakakertaPerkasa Inakakerta2022
11,13,PT Wahana Baratama Mining,13,6.0,6,Wahana Baratama,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Selatan,Satui,PT Wahana Baratama MiningWahana Baratama2022
12,14,PT Teguh Sinarabadi,14,4.0,4,Teguh Sinar Abadi/Firman Ketaun Perkasa,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Barat,PT Teguh SinarabadiTeguh Sinar Abadi/Firman Ke...
13,15,PT Firman Ketaun Perkasa,15,4.0,4,Teguh Sinar Abadi/Firman Ketaun Perkasa,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kalimantan Timur,Kutai Barat,PT Firman Ketaun PerkasaTeguh Sinar Abadi/Firm...
14,27,PT Lahai Coal,27,9.0,9,Haju,2021,NaN,NaN,NaN,NaN,0.10,0.40,3,1.5,0,Kalimantan Tengah,NaN,PT Lahai CoalHaju2021
15,27,PT Lahai Coal,27,14.0,14,Bara,2021,NaN,NaN,NaN,NaN,0.30,4.00,10.6,7.6,3.2,Kalimantan Tengah,NaN,PT Lahai CoalBara2021
16,29,PT Kalteng Coal,29,10.0,10,Luon,2021,NaN,NaN,NaN,NaN,6.90,19.30,24.7,0,17.7,Kalimantan Tengah,NaN,PT Kalteng CoalLuon2021


In [12]:
new_cc_ccp_ms_df.columns

Index(['name_x', 'company_id', 'name_y', 'year_measured', 'calorific_value',
       'production_volume', 'overburden_removal_volume', 'strip_ratio',
       'resources_inferred', 'resources_indicated', 'resources_measured',
       'reserves_proved', 'reserves_probable', 'province', 'city',
       'nameyearcompany'],
      dtype='object')

In [ ]:
def safe_value(val):
    return None if pd.isna(val) else val

for _, row in new_cc_ccp_ms_df.iterrows():

    q_company = Company.select().where(Company.name == row['name_x']).first()
    q_company_id = q_company.id if q_company else None

    mining_site = MiningSite(
        name=safe_value(row['name_y']),
        year=safe_value(row['year_measured']),
        company=q_company_id,
        calorific_value=safe_value(row['calorific_value']),
        production_volume=safe_value(row['production_volume']),
        overburden_removal_volume=safe_value(row['overburden_removal_volume']),
        strip_ratio=safe_value(row['strip_ratio']),
        province=safe_value(row['province']),
        city=safe_value(row['city'])
    )

    # mining_site.save()

In [10]:
fields_to_compare = [
    'name',
    'idx_ticker',
    'operation_province',
    'operation_kabkot',
    'representative_address',
    'company_type',
    'key_operation',
    'activities',
    'website',
    'phone_number',
    'email'
]

for _, row in existing_company_df.iterrows():
    company = Company.get_or_none(Company.name == row['name'])
    if company:
        differences = []
        for field in fields_to_compare:
            model_value = getattr(company, field)
            df_value = safe_value(row[field])

            if (model_value != df_value) and (df_value is not None):
                differences.append([field, model_value, df_value])
                # setattr(company, field, df_value) 

        if differences:
            # company.save()
            print(f"\nDifferences for company '{company.name}':")
            print(tabulate(differences, headers=["Field", "DB Value", "CSV Value"], tablefmt="grid"))



Differences for company 'PT Pada Idi':
+------------------+--------------+--------------+
| Field            | DB Value     | CSV Value    |
+==================+==============+==============+
| operation_kabkot | Barito Utara | North Barito |
+------------------+--------------+--------------+

Differences for company 'PT Kaltim Prima Coal':
+------------------+------------+-------------+
| Field            | DB Value   | CSV Value   |
+==================+============+=============+
| operation_kabkot |            | Kutai Timur |
+------------------+------------+-------------+
| phone_number     | 549-521155 | 054-9521155 |
+------------------+------------+-------------+

Differences for company 'Bumi Resources Minerals Tbk':
+------------------------+------------+---------------------------------------------------------------------------------------------------------------------+
| Field                  | DB Value   | CSV Value                                                         